<a href="https://colab.research.google.com/github/totminaekaterina/Coursework_multitasking_learning/blob/main/QG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
import json

# Загрузка датасета
with open('train.json', 'r', encoding='utf-8-sig') as f:
    data = json.load(f)

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rut5-base-multitask")

model = AutoModelForSeq2SeqLM.from_pretrained("cointegrated/rut5-base-multitask")

/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
from transformers import AdamW

In [6]:
# Определение параметров обучения
num_epochs = 10
batch_size = 4
learning_rate = 3e-5

In [7]:
from sklearn.metrics import f1_score, accuracy_score

In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
device

device(type='cuda')

In [9]:
# Определение оптимизатора и функции потерь
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# Обучение модели
for epoch in range(num_epochs):
    epoch_loss = 0.0
    num_batches = len(data) // batch_size

    # добавленные строки
    f1_score_total = 0.0
    acc_total = 0.0

    for i in range(num_batches):
        batch = data[i * batch_size: (i+1) * batch_size]

        # Токенизация данных и генерация вопросов
        input_sentences = [item['passage'] for item in batch]
        target_questions = [item['question'] for item in batch]
        input_tokens = tokenizer.batch_encode_plus(input_sentences, padding=True, truncation=True, return_tensors='pt')
        target_tokens = tokenizer.batch_encode_plus(target_questions, padding=True, truncation=True, return_tensors='pt')

        # Подготовка входных данных
        input_ids = input_tokens['input_ids'].to(device)
        attention_mask = input_tokens['attention_mask'].to(device)
        target_ids = target_tokens['input_ids'].to(device)
        target_ids[target_ids == tokenizer.pad_token_id] = -100

        # Обнуление градиентов и вычисление предсказаний модели
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids)

        # Расчет функции потерь и обновление весов
        loss = outputs.loss
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()

        # добавленные строки
        predictions = outputs.logits.argmax(dim=-1)
        f1_score_total += f1_score(target_ids.view(-1).cpu().numpy(), predictions.view(-1).cpu().numpy(), average='macro')
        acc_total += accuracy_score(target_ids.view(-1).cpu().numpy(), predictions.view(-1).cpu().numpy())

    print('Epoch:', epoch+1, '\tLoss:', epoch_loss/num_batches, '\tF1 Score:', f1_score_total/num_batches, '\tAccuracy:', acc_total/num_batches)

# Сохранение обученной модели
model.save_pretrained('question_generation_model')

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Epoch: 1 	Loss: 1.5903467690099022 	F1 Score: 0.4832782734093054 	Accuracy: 0.5638025941215289
Epoch: 2 	Loss: 0.9442548288087823 	F1 Score: 0.6149415500739588 	Accuracy: 0.6592435032086709
Epoch: 3 	Loss: 0.59163909305731 	F1 Score: 0.7151062160071894 	Accuracy: 0.7194329419660993
Epoch: 4 	Loss: 0.3658997725857341 	F1 Score: 0.7936717967177362 	Accuracy: 0.7626428663480311
Epoch: 5 	Loss: 0.23037898378953917 	F1 Score: 0.8427806384795609 	Accuracy: 0.7882159879224754
Epoch: 6 	Loss: 0.15507449073508142 	F1 Score: 0.8710709438144962 	Accuracy: 0.8020568602527078
Epoch: 7 	Loss: 0.10408216320485457 	F1 Score: 0.8923608545167833 	Accuracy: 0.8118036941864307
Epoch: 8 	Loss: 0.0743196296289316 	F1 Score: 0.9050667039857796 	Accuracy: 0.8180069809621445
Epoch: 9 	Loss: 0.0630970623397995 	F1 Score: 0.9105401040325845 	Accuracy: 0.8199626023097651
Epoch: 10 	Loss: 0.05112106864073141 	F1 Score: 0.9131229454109415 	Accuracy: 0.8212121849369416


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Сохранение обученной модели
model.save_pretrained('/content/drive/MyDrive/Bert/question_generation_model_1')

In [12]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [13]:
# Инициализация токенизатора и модели
model = AutoModelForSeq2SeqLM.from_pretrained('/content/question_generation_model_1')

In [14]:
import json

In [17]:
with open('/content/drive/MyDrive/Bert/TrainOnlySentenceJson.json', 'r', encoding='utf-8-sig',) as f:
    test_data = json.load(f)

print(test_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
input_sentences = [item['text'] for item in test_data]
input_tokens = tokenizer.batch_encode_plus(input_sentences, padding=True, truncation=True, return_tensors='pt')

In [ ]:
output_tokens = model.generate(
    input_ids=input_tokens['input_ids'].to('cpu'),
    attention_mask=input_tokens['attention_mask'].to('cpu'),
    max_length=50,
    num_beams=4,
    early_stopping=True
)

In [ ]:
generated_questions = [tokenizer.decode(ids, skip_special_tokens=True) for ids in output_tokens]

In [ ]:
output_data = []
for i in range(len(test_data)):
    output_data.append({'Sentence': test_data[i]['text'], 'Question': generated_questions[i]})

In [ ]:
with open('/content/drive/MyDrive/Bert/output_data_big.json', 'w', encoding='utf-8') as f:
    json.dump(output_data, f, ensure_ascii=False, indent=4)